In [2]:
!pip install diffusers

/Users/tanishrajput/.zshenv:1: bad assignment
  Using cached diffusers-0.32.2-py3-none-any.whl.metadata (18 kB)
Using cached diffusers-0.32.2-py3-none-any.whl (3.2 MB)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
!pip install accelerate

/Users/tanishrajput/.zshenv:1: bad assignment

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

/Users/tanishrajput/.zshenv:1: bad assignment

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
/Users/tanishrajput/.zshenv:1: bad assignment
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub

In [3]:
from diffusers import StableDiffusionImg2ImgPipeline
import torch
from PIL import Image
import io
import matplotlib.pyplot as plt
import time
import ipywidgets as widgets
from IPython.display import display

# Load model function
def load_model():
    model_id = "nitrosocke/Ghibli-Diffusion" # Provide the actual model ID here
    dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    print("Loading model...")
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=dtype)
    pipe.to("cuda" if torch.cuda.is_available() else "cpu")
    pipe.enable_attention_slicing()  # Optimize memory usage
    print("Model loaded!")
    return pipe

# Function to generate Ghibli-style image
def generate_ghibli_image(image, pipe, strength):
    image = image.convert("RGB")
    width, height = image.size
    
    # Resize while maintaining aspect ratio
    ratio = min(512 / width, 512 / height)
    new_width = int(width * ratio)
    new_height = int(height * ratio)
    
    image = image.resize((new_width, new_height))
    
    prompt = ""
    print("Generating image...")
    start_time = time.time()
    result = pipe(prompt=prompt, image=image, strength=strength).images[0]
    print(f"Image generated in {time.time() - start_time:.2f} seconds!")
    return result

# Check for GPU
gpu_info = "GPU is available!" if torch.cuda.is_available() else "Warning: GPU not available. Processing will be slow."
print(gpu_info)

# Load the model
pipe = load_model()

# File upload widget
upload_widget = widgets.FileUpload(accept="image/*", multiple=False)

def on_upload_change(change):
    uploaded_file = list(upload_widget.value.values())[0]
    image = Image.open(io.BytesIO(uploaded_file['content']))
    
    # Display original image
    plt.figure(figsize=(5, 5))
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')
    plt.show()
    
    # Ask for strength input
    strength = float(input("Enter stylization strength (0.3-0.8, recommended 0.6): "))
    strength = max(0.3, min(0.8, strength))  # Clamp between 0.3 and 0.8
    
    # Generate and display the result
    result_img = generate_ghibli_image(image, pipe, strength)
    
    plt.figure(figsize=(5, 5))
    plt.imshow(result_img)
    plt.title("Ghibli Portrait")
    plt.axis('off')
    plt.show()
    
    # Save the output image
    output_filename = "ghibli_portrait.png"
    result_img.save(output_filename)
    print(f"Image saved as {output_filename}!")

# Display the upload widget
display(upload_widget)
upload_widget.observe(on_upload_change, names="value")


Loading model...


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

diffusion_pytorch_model.safetensors:   6%|6         | 210M/3.44G [00:00<?, ?B/s]

model.safetensors:  45%|####4     | 220M/492M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:  63%|######2   | 210M/335M [00:00<?, ?B/s]

model.safetensors:  16%|#6        | 199M/1.22G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Model loaded!


FileUpload(value={}, accept='image/*', description='Upload')